<a href="https://colab.research.google.com/github/Vamsi-Malineni/Research-work/blob/master/pinn_trial_tf2_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf
import numpy as np
import tensorflow_probability as tfp 
import os
import sys
import scipy.io
import time 


In [ ]:
class pinn(object):
  def __init__(self,layers,optimizer,ub,lb):
    
    self.model=tf.keras.Sequential()
    
    self.model.add(tf.keras.layers.InputLayer(input_shape=(layers[0],)))
    
    self.model.add(tf.keras.layers.Lambda(lambda X: 2.0*(X-lb)/(ub-lb)-1))

    for width in layers[1:]:
      self.model.add(tf.keras.layers.Dense(
          width,activation=tf.nn.tanh,
          kernel_initializer='glorot_normal'
      ))

  # finding the sizes of weights and biases for each layer in the neural network  
    self.sizes_w=[]
    self.sizes_b=[]

    for i,width in enumerate(layers):
      if i!=1:
        self.sizes_w.append(int(width*layers[1]))
        self.sizes_b.append(int(width if i!=0 else layers[1]))
    self.dtype=tf.float32

  # Defining two variables that have to be learnt (lambda1,lambda2)
    self.lambda1=tf.Variable([0.0],dtype=self.dtype)
    self.lambda2=tf.Variable([0.0],dtype=self.dtype)

    self.optimizer=optimizer

  def get_parameters(self,numpy=False):
    l1=self.lambda1
    l2=self.lambda2
    if numpy :
      return l1.numpy()[0],l2.numpy()[0]
    return l1,l2

  def setup_pinn(self,xtrain,ytrain,ttrain):
    l1,l2=self.get_parameters()
    # Inputs to be watched by gradient tape
    x=tf.convert_to_tensor(xtrain,dtype=self.dtype)
    y=tf.convert_to_tensor(ytrain,dtype=self.dtype)
    t=tf.convert_to_tensor(ttrain,dtype=self.dtype)
    
    with tf.GradientTape(persistent=True) as g:
      g.watch(x)
      g.watch(y)
      g.watch(t)

      